# Modelowanie liczby roszczeń

### Generowanie niejednorodnego procesu Poissona metodą przerzedzania

In [1]:
import numpy as np
import pandas as pd

In [53]:
def nhpp(lambda_func,lambda_func_params, t_max):
    """
    Funkcja generuje niejednorodny proces Poissona metodą przerzerzedzania.

    Argumenty:
        lambda_func: funkcja
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności
        t_max: float
            Maksymalny czas symulacji
        lambda_max: float
            Ograniczenie górne funkcji lambda_func
    Zwraca:
        numpy.ndarray
            Tablica czasów.
    """

    S = 0
    N = 0
    S_tab = []
    lambda_max = lambda_func(0,lambda_func_params)[1]
    
    while S < t_max:
        U = np.random.uniform(0,1)
        S=S+(-1/lambda_max)*np.log(U)
        if S > t_max:
            break
        U = np.random.uniform(0,1)
        if U <= lambda_func(S,lambda_func_params)[0]/lambda_max:
            N = N + 1
            S_tab.append(S)

    return N, S_tab

In [3]:
def sin_int_fun(t,lambda_func_params):
    """
    Sinusoidalna funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności.
            W tym przypadku listę zawierającą: [lambda_0, lambda_1, T, phi]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0+lambda_1.    
    """
    return lambda_func_params[0]+lambda_func_params[1]*np.sin(2*np.pi*t/lambda_func_params[2]+lambda_func_params[3]), lambda_func_params[0]+lambda_func_params[1]

In [4]:
def exp_int_fun(t,lambda_func_params):
    """
    Wykładnicza funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności. 
            W tym przypadku listę zawierającą: [lambda_0, lambda]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0/lambda.
    """
    return lambda_func_params[0]*np.exp(-lambda_func_params[1]*t), lambda_func_params[0]*lambda_func_params[1]

In [5]:
nhpp(sin_int_fun,[1,3,1,0],10)

(15,
 [0,
  0.25205028516484734,
  0.3402615958007469,
  1.1190269902610375,
  1.1493929651402246,
  1.1504309554728027,
  2.5427945572335156,
  3.399393787586424,
  5.278311768972828,
  6.1372554158009,
  6.467015289000762,
  8.225260005023692,
  9.096511816201756,
  9.164577027574243,
  9.29866488745965,
  9.350839188252944])

In [6]:
nhpp(exp_int_fun,[2,1/10],100)

(6,
 [0,
  1.994897765072185,
  2.9032915339971916,
  6.046441729221631,
  14.713296979146534,
  36.31488305456054,
  54.31099339815704])

Empiryczny proces $N_t$.

In [7]:
data = pd.read_csv("https://github.com/ndzadz/mgr/blob/main/data_us_1990_2022.csv?raw=true",encoding="latin1",sep=";",header=0)
p95 = np.percentile(data[data["Insured Damages, Adjusted (\'000 US$)"].isna()==0]["Insured Damages, Adjusted (\'000 US$)"],95)
data = data[(data["Insured Damages, Adjusted (\'000 US$)"].isna()==0)&(data["Insured Damages, Adjusted (\'000 US$)"]<p95)]
data = data.rename(columns={'ï»¿Dis No':'Dis No'})

In [97]:
N = pd.Series(data.groupby('Year').count()['Dis No'])
N.index = range(1,len(N)+1)
N

1     17
2     28
3     29
4     25
5     11
6     18
7     12
8     26
9     26
10    16
11     7
12    10
13    13
14    13
15    13
16     9
17    12
18    22
19    13
20    10
21     9
22    16
23    20
24    20
25    15
26    25
27    21
28    32
29     4
30    10
31     8
32     1
33     2
Name: Dis No, dtype: int64

In [102]:
def MSE(N, N_pred):
    n = len(N)
    return np.sum(np.array((N-N_pred)**2))/n

In [103]:
def MAE(N, N_pred):
    n = len(N)
    return np.sum(np.abs((N-N_pred)))/n

Metoda najmniejszej sumy kwadratów

In [10]:
from scipy.optimize import minimize

Sinusoidalna funkcja intensywności

In [11]:
def sum_of_squares_sin(lambda_func_params):
    t = np.array(range(len(N)))
    return np.sum(np.array((N-sin_int_fun(t, lambda_func_params)[0])**2))

In [12]:
sum_of_squares_sin([1,3,1,0])

9015.000000000011

In [50]:
bnds = ((0, None), (0, None), (0, None), (0, None)) # ograniczenia na wartości parametrów
params = [1.0, 1.0, 1.0, 1.0] # Przyjmujemy początkowe wartości parametrów
result = minimize(sum_of_squares_sin, params, bounds=bnds, method='L-BFGS-B')
print(result)


      fun: 1575.267861512148
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.27373657e-05, -2.27373677e-05,  7.44648792e-02, -1.81898925e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 290
      nit: 44
     njev: 58
   status: 0
  success: True
        x: array([14.51045062,  6.56946102,  1.0560816 ,  9.69327362])


Symulacja procesu z sinusoidalną funkcją intensywności dla wyliczonych parametrów

In [78]:
n = nhpp(sin_int_fun,result.x,33)
print(n)

(492, [0.004787051796027816, 0.015455299687550602, 0.042511903980397135, 0.08081770505974706, 0.34068807344290575, 0.5883670527172395, 0.6327877926051018, 0.7121274660953353, 0.7482737414281623, 0.7707550995918624, 0.7784330000962786, 0.7859017751463567, 0.8258091208184652, 0.8375679792739078, 0.8793591031050422, 0.9059002063038999, 0.9295545081347651, 0.9403827008749873, 1.0477152243334023, 1.0608506425926052, 1.0914433039016953, 1.181846844378093, 1.192261595678565, 1.1934165064591844, 1.218421640025052, 1.4804375214746477, 1.5028614652809886, 1.5453071226604602, 1.57615023786629, 1.65109594576342, 1.6745744903793667, 1.6882171978539926, 1.7275165702295474, 1.8142306423681867, 1.8199712705979134, 1.8265782057704711, 1.922006340325993, 1.9317406551985978, 1.9802501189414625, 2.066040079064092, 2.4259188098290245, 2.436720727112766, 2.5534474034138133, 2.6280751769420125, 2.6492748011784557, 2.7245175847006933, 2.7487814557350556, 2.848483124094861, 2.85732711393831, 3.001623336117045,

In [105]:
unique, counts = np.unique(np.ceil(n[1]), return_counts=True)
values = pd.DataFrame({'Year':unique, 'N':counts})
index_df = pd.DataFrame({'Year':np.array(range(1,34))})
merged_table = pd.merge(index_df, values, on='Year', how='left').fillna(0).set_index('Year')
N_pred = pd.Series(merged_table['N'])
N_pred

Year
1     18
2     21
3     10
4     10
5     17
6     15
7     11
8     11
9      9
10    17
11    12
12    22
13    18
14    13
15     9
16    19
17    14
18    12
19    13
20    13
21    13
22    14
23    14
24    21
25    14
26    19
27    14
28    14
29    17
30    20
31    18
32    12
33    18
Name: N, dtype: int64

In [107]:
print("MSE: ", MSE(N,N_pred), "MAE: ", MAE(N,N_pred))

MSE:  85.84848484848484 MAE:  7.303030303030303


Wykładnicza funkcja intensywności

In [16]:
def sum_of_squares_exp(lambda_func_params):
    t = np.array(range(len(N)))
    return np.sum(np.array((N-exp_int_fun(t, lambda_func_params)[0])**2))

In [17]:
sum_of_squares_exp([1,3])

10006.16344218761

In [18]:
bnds = ((0, None), (0, None)) # ograniczenia na wartości parametrów
params = [1.0, 1.0] # Przyjmujemy początkowe wartości parametrów
result = minimize(sum_of_squares_exp, params, bounds=bnds, method='L-BFGS-B')
print(result)


      fun: 2130.1683437993447
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0001819,  0.0721684])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 51
      nit: 9
     njev: 17
   status: 0
  success: True
        x: array([1.85912816e+01, 1.31208572e-02])


Symulacja procesu z sinusoidalną funkcją intensywności dla wyliczonych parametrów

In [108]:
n = nhpp(exp_int_fun,result.x,33)
print(n)

(3, [0.0660809419884032, 0.1430192006385102, 0.16658938945024782])


In [111]:
unique, counts = np.unique(np.ceil(n[1]), return_counts=True)
values = pd.DataFrame({'Year':unique, 'N':counts})
index_df = pd.DataFrame({'Year':np.array(range(1,34))})
merged_table = pd.merge(index_df, values, on='Year', how='left').fillna(0).set_index('Year')
N_pred = pd.Series(merged_table['N'])
N_pred

Year
1     3.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
Name: N, dtype: float64

In [112]:
print("MSE: ", MSE(N,N_pred), "MAE: ", MAE(N,N_pred))

MSE:  300.42424242424244 MAE:  15.454545454545455
